In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, OpenAI
from openai import OpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import json
from pathlib import Path
from langchain.vectorstores import Qdrant
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ["COHERE_API_KEY"] = ''
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = ""

In [ ]:
reader = "../test data/sample 1.pdf"
loader = UnstructuredPDFLoader(reader)
data = loader.load()
for doc in data:
    data = doc.page_content

In [ ]:
url = 'http://localhost:6333'
collection_name="Sales_Team",

In [ ]:
text_split = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)

mesg_chunk = text_split.split_documents(data)

In [ ]:
vectorstore = Qdrant.from_documents(
  mesg_chunk,
  url=url,
  collection_name=collection_name,
  embedding = OpenAIEmbeddings()
)

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
prompt

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 20})


In [ ]:
# look at the 20 retrived documents
retriever.invoke("what is the name of the candidate?")

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [ ]:
llm = ChatOpenAI()

In [ ]:
from operator import itemgetter
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [ ]:
contextual_compression_retrieval_chain.invoke({"question" : "what is the name of the candidate?"})["response"].content